# Toycode for PyTorch-Lightning

So far I have bumped into many technical difficulties related to Pytorch-lightning and Tensorboard. In this notebook, I will use a simple model and dataset to experiment with different functionalities including: 

<ul>
    <li>Tensorboard logging</li>
    <li>Callback</li> 
    <li>Freeze parameters</li>
</ul>

Other things I need to experiment 

<ul>
    <li>Rouge metric and other metrics</li>
    <li>Optimizer and scheduler</li>
</ul>

## Resources 

<ul>
    <li><a href="https://pytorch-lightning.readthedocs.io/en/stable/">PyTorch-lightning doc</a></li>
    <li><a href="https://pytorch-lightning.readthedocs.io/en/stable/rapid_prototyping_templates.html">Rapid prototyping</a></li>
</ul>
    

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from transformers import AdamW

In [2]:
# Generate 1000 points
X = (np.random.rand(1000, 5) ** 2 * 5).tolist()
# Labels are the norms of 1000 points
y = (np.sqrt(np.sum(np.square(X), axis = 1))).tolist()

In [3]:
class MyDataset(Dataset): 
    def __init__(self, X, y): 
        super().__init__()
        self.X = X
        self.y = y
        
    def __len__(self): 
        return len(y)
    
    def __getitem__(self,idx):
        return {
            'source': torch.tensor(X[idx]), 
            'target': torch.tensor(y[idx])
        }

In [7]:
class MyModel(pl.LightningModule): 
    ''' Part 1: Define the architecture of model in init '''
    def __init__(self, hparams):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(5, 10)
        self.layer2 = nn.Linear(10, 8)
        self.layer3 = nn.Linear(8, 1)
        self.hparams = hparams 
        
    ''' Part 2: Define the forward propagation '''
    def forward(self, x): 
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x
    
    ''' Part 3: Prepare optimizer and scheduler '''
    def configure_optimizers(self): 
        optimizer = AdamW(self.parameters(), lr = self.hparams['learning_rate'])
        return optimizer
    
    ''' Part 4.1: Training logic '''
    def training_step(self, batch, batch_idx): 
        X = batch['source']
        y = batch['target']
        y_hat = self(X)    # Calls forward function 
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    ''' Part 4.2: Validation logic '''
    def validation_step(self, batch, batch_idx): 
        X = batch['source']
        y = batch['target']
        y_hat = self(X)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss)
        
    ''' Part 4.3: Test logic '''
    def test_step(self, batch, batch_idx): 
        X = batch['source']
        y = batch['target']
        y_hat = self(X)
        loss = F.mse_loss(y_hat, y)
        self.log('test_loss', loss)
        
    ''' Part 5: Data loaders '''
    def train_dataloader(self): 
        dataset = MyDataset(X[:800], y[:800])
        return DataLoader(dataset, batch_size = hparams['batch_size'])
    
    def val_dataloader(self): 
        dataset = MyDataset(X[800:950], y[800:950])
        return DataLoader(dataset, batch_size = hparams['batch_size'])
    
    def test_dataloader(self): 
        dataset = MyDataset(X[950:], y[950:])
        return DataLoader(dataset, batch_size = hparams['batch_size'])

In [8]:
hparams = {
    'learning_rate': 3e-4, 
    'batch_size': 16
}

In [9]:
model = MyModel(hparams)
trainer = pl.Trainer(gpus = 1, max_epochs = 3, progress_bar_refresh_rate = 20)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 60    
1 | layer2 | Linear | 88    
2 | layer3 | Linear | 9     


<ipython-input-7-e01a0e19433f>:36: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


<ipython-input-7-e01a0e19433f>:27: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)
<ipython-input-7-e01a0e19433f>:27: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


<ipython-input-7-e01a0e19433f>:36: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


1

In [10]:
trainer.test()

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(23.9610, device='cuda:0'),
 'train_loss': tensor(21.2212, device='cuda:0'),
 'val_loss': tensor(23.9610, device='cuda:0')}
--------------------------------------------------------------------------------



<ipython-input-7-e01a0e19433f>:44: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)
<ipython-input-7-e01a0e19433f>:44: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


[{'train_loss': 21.221168518066406,
  'val_loss': 23.96100616455078,
  'test_loss': 23.96100616455078}]

In [19]:
model(torch.tensor([1.,1.,2.,2.,3.]).to('cuda'))

tensor([0.1921], device='cuda:0', grad_fn=<AddBackward0>)

In [20]:
X[0]

[0.007745272301294734,
 0.6838872275291756,
 1.2173503655308444,
 0.0007588582869603618,
 2.0238172648539745]

In [21]:
y[0]

2.458768093795258

In [22]:
model(torch.tensor([0.007745272301294734,
 0.6838872275291756,
 1.2173503655308444,
 0.0007588582869603618,
 2.0238172648539745]).to('cuda'))

tensor([0.1155], device='cuda:0', grad_fn=<AddBackward0>)

In [26]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/version_2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 21972), started 0:01:45 ago. (Use '!kill 21972' to kill it.)